In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import joblib as jb
import helper_functions as hlf

plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
%matplotlib inline

Using TensorFlow backend.


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential, load_model, Model
from keras.utils import to_categorical
from keras.preprocessing import image

# Load Training Data

In [ ]:
def load_data(damselpath, dragonpath, start, end):
    '''
    This function loads a pre-saved numpy arrays with their labels
    load_data(damselpath, dragonpath, start, end):
    Input:
        damselpath: Damselflies numpy location
        dragonpath: Dragonflies numpy location
        start: Integer for slicing the arrays
        end: Integer for slicing the arrays
    Returns:
        Retuns a concatenated numpy array with dragonfly and damselfly images (dragons first),
        and a vector the size of the image vector where dragons value is 1 and damsel 0
    '''
    data = np.concatenate((np.load(dragonpath)[start:end], np.load(damselpath)[start:end]), axis=0)
    dragon_images = len(np.load(dragonpath)[start:end])
    labels = labels_vector(data, dragon_images)                           
    return data, labels

In [ ]:
def labels_vector(data, dragon_images):
    '''
    This function creates a binary vector
    labels_vector(data, dragon_images):
    Input:
        data: List containing the images in numpy array format
        dragon_images: The the amount of dragonfly images in the array
    Returns:
        This function returns a binary vector the length of the given data vector.
        The function assumes the vector is order dragonfly images come first
    '''
    labels = np.zeros(len(data))
    labels[:dragon_images] = 1
    return labels

In [ ]:
def create_model(weights_path=None, weights_name=None):
    '''
    This function create a sequantial model with input shape of (256 256, 1)
    create_model(weights_path=None, weights_name=None):
    Input:
        weights_path: The full path to the model weights directory. default=None
        weights_name: Weights file name. default=None
    Returns:
        This model returns a compiled sequantial model with input shape (256 256, 1)
        and output shape of 2.
    
    '''
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='tanh',
                     input_shape=(256, 256, 1)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))
    
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.4))
    
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.4))

    model.add(Dense(2, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
    
    if weights_path and weights_name:
        model.load_weights(f'{weights_path}{weights_name}')
    return model

In [ ]:
def create_train_validation(training_data, labels):
    '''
    This function seprates the images and labels to training and validation sets
    create_train_validation(training_data, labels):
    Input:
        training_data: Images in numpy array format
        labels: Vector with the image labels
    Returns:
        Shuffled, reshaped arrays with matching labels
    '''
    # split to sets
    x_train, x_val, y_train, y_val = train_test_split(training_data, labels, test_size=0.05)
    
    # adjust variables dimensions to fit into the cnn    
    x_train = np.expand_dims(x_train, axis=-1)
    x_val = np.expand_dims(x_val, axis=-1)
    
    y_train_cat = to_categorical(y_train)
    y_val_cat = to_categorical(y_val)
    return x_train, x_val, y_train, y_val, y_train_cat, y_val_cat

In [7]:
start, end = 0, 2000

In [11]:
# load the images to train the clasifier
damsel_train_path = r'D:\Linnaeus_models\damsel\train\damsel_train.npy'
dragon_train_path = r'D:\Linnaeus_models\dragon\train\dragon_train.npy'

training_data, labels = hlf.load_data(damsel_train_path, dragon_train_path, start, end)

In [12]:
# create training/validation sets
x_train, x_val, y_train, y_val, y_train_cat, y_val_cat = hlf.create_train_validation(training_data, labels)

In [13]:
# create and compile keras seqeuntial model
model = hlf.create_model()
model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 32)      320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 252, 252, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 126, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 126, 126, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 124, 124, 64)      18496     
__________________________________________________________

In [ ]:
train_history = model.fit(x_train, y_train, epochs=10, batch_size=180,
                          validation_data=(x_val, y_val)
                         ) 
model.save_weights('./model_weights/first2000')

In [ ]:
variable =  './train_history/first2000'
jb.dump(train_history, variable)

### Reset Kernel 
 * Load previous model weights
 * Re-train on unseen images

In [14]:
# Reset kernel, Load past model and retrain on the model on new images
weights_path = './model_weights/'
weights_name = 'fifth2000'
pre_trained_model = hlf.create_model(weights_path, weights_name)

In [ ]:
# load new images and create new train/validation sets
start, end = 12000, 12694

damsel_train_path = r'D:\Linnaeus_models\damsel\train\damsel_train.npy'
dragon_train_path = r'D:\Linnaeus_models\dragon\train\dragon_train.npy'

training_data, labels = hlf.load_data(damsel_train_path, dragon_train_path, start, end)

x_train, x_val, y_train, y_val, y_train_cat, y_val_cat = hlf.create_train_validation(training_data, labels)

In [ ]:
train_history = pre_trained_model.fit(x_train, y_train_cat, epochs=2, batch_size=35,
                                      validation_data=(x_val, y_val_cat)
                                     ) 
pre_trained_model.save_weights('./model_weights/sixth2000')
history_path = './train_history/sixth2000'
jb.dump(train_history, history_path)

In [ ]:
loss, acc = pre_trained_model.evaluate(x_val, to_categorical(y_val))
print(f'Validation accuracy: {acc * 100}%')

In [ ]:
preds = pre_trained_model.predict(x_val)

preds_class = pre_trained_model.predict_classes(x_val)

reverted_x = np.squeeze(x_val, axis=-1)

In [ ]:
plt.imshow(reverted_x[1], cmap='gray')

In [ ]:
# load testing set

In [ ]:
# Reset kernel, Load past model and retrain on the model on new images
weights_path = './model_weights/'
weights_name = 'seventh694'
pre_trained_model = hlf.create_model(weights_path, weights_name)

In [ ]:
start, end = 0, -1
# load the images to train the clasifier
damsel_test_path = r'D:\Linnaeus_models\damsel\test\damsel_test.npy'
dragon_test_path = r'D:\Linnaeus_models\dragon\test\dragon_test.npy'

test_set, labels = hlf.load_data(damsel_test_path, dragon_test_path, start, end)

In [ ]:
preds = pre_trained_model.predict(np.expand_dims(test_set, axis=-1))

In [ ]:
preds_class = pre_trained_model.predict_classes(np.expand_dims(test_set, axis=-1))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score

In [ ]:
accuracy_score(labels, preds_class)

In [ ]:
accuracy_score(labels, preds_class) # 87.5 6th model weights

# Previouse Model Architectures